In [1]:
import pandas as pd 
import numpy as np
from fbprophet import Prophet
import plotly
import plotly.graph_objects as go

/Users/jisha/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
rental = pd.read_csv('csv/Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv')

In [3]:
rental.head()

,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02
0,102001,United States,0,1325.0,1331.0,1338.0,1350.0,1370.0,1385,1396,...,1701,1703,1703,1695,1691,1686,1694,1693,1701,1708
1,394913,"New York, NY",1,2282.0,2300.0,2304.0,2336.0,2370.0,2392,2406,...,2712,2696,2666,2601,2551,2496,2472,2451,2472,2471
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,1735.0,1761.0,1767.0,1794.0,1832.0,1863,1888,...,2452,2458,2457,2452,2449,2451,2464,2459,2463,2472
3,394463,"Chicago, IL",3,1413.0,1423.0,1426.0,1442.0,1464.0,1480,1489,...,1690,1691,1685,1672,1654,1636,1628,1624,1622,1634
4,394514,"Dallas-Fort Worth, TX",4,1134.0,1127.0,1137.0,1154.0,1174.0,1190,1198,...,1480,1485,1491,1489,1491,1492,1505,1510,1507,1519


In [4]:
rental.shape

(106, 89)

In [5]:
pd.set_option('display.max_rows', 10)

In [6]:
rental.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
2014-01       5
2014-02       1
             ..
2020-10       0
2020-11       0
2020-12       0
2021-01       0
2021-02       0
Length: 89, dtype: int64

In [7]:
rental.columns

Index(['RegionID', 'RegionName', 'SizeRank', '2014-01', '2014-02', '2014-03',
       '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09',
       '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03',
       '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09',
       '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03',
       '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09',
       '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03',
       '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
       '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
       '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
       '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03',
       '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
       '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03',
       '2020-04', '2020-05', '2020-06', '2020-

In [8]:
rent_melt = (rental.melt(id_vars= ['RegionID', 'RegionName', 'SizeRank'],
                         var_name = 'ds',
                         value_name = 'y')
                         ).reset_index(drop=True)

In [10]:
rent_melt.head()

,RegionID,RegionName,SizeRank,ds,y
0,102001,United States,0,2014-01,1325.0
1,394913,"New York, NY",1,2014-01,2282.0
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01,1735.0
3,394463,"Chicago, IL",3,2014-01,1413.0
4,394514,"Dallas-Fort Worth, TX",4,2014-01,1134.0


In [11]:
print(rent_melt.RegionName.nunique())
rent_melt.RegionName.unique()

106


array(['United States', 'New York, NY',
       'Los Angeles-Long Beach-Anaheim, CA', 'Chicago, IL',
       'Dallas-Fort Worth, TX', 'Philadelphia, PA', 'Houston, TX',
       'Washington, DC', 'Miami-Fort Lauderdale, FL', 'Atlanta, GA',
       'Boston, MA', 'San Francisco, CA', 'Detroit, MI', 'Riverside, CA',
       'Phoenix, AZ', 'Seattle, WA', 'Minneapolis-St Paul, MN',
       'San Diego, CA', 'St. Louis, MO', 'Tampa, FL', 'Baltimore, MD',
       'Denver, CO', 'Pittsburgh, PA', 'Portland, OR', 'Charlotte, NC',
       'Sacramento, CA', 'San Antonio, TX', 'Orlando, FL',
       'Cincinnati, OH', 'Cleveland, OH', 'Kansas City, MO',
       'Las Vegas, NV', 'Columbus, OH', 'Indianapolis, IN',
       'San Jose, CA', 'Austin, TX', 'Virginia Beach, VA',
       'Nashville, TN', 'Providence, RI', 'Milwaukee, WI',
       'Jacksonville, FL', 'Memphis, TN', 'Oklahoma City, OK',
       'Louisville-Jefferson County, KY', 'Hartford, CT', 'Richmond, VA',
       'New Orleans, LA', 'Buffalo, NY', 'Raleig

In [12]:
cities_list = list(rental['RegionName'])

In [13]:
len(cities_list)

106

In [14]:
def rnd_series(city):
    subset = rent_melt[rent_melt['RegionName']== city]
    dates = (pd.DataFrame({'ds': pd.to_datetime(rent_melt['ds'])}))
    return subset

In [15]:
series = [rnd_series(city) for city in cities_list]

In [16]:
len(series)

106

In [17]:
series[0]

,RegionID,RegionName,SizeRank,ds,y
0,102001,United States,0,2014-01,1325.0
106,102001,United States,0,2014-02,1331.0
212,102001,United States,0,2014-03,1338.0
318,102001,United States,0,2014-04,1350.0
424,102001,United States,0,2014-05,1370.0
...,...,...,...,...,...
8586,102001,United States,0,2020-10,1686.0
8692,102001,United States,0,2020-11,1694.0
8798,102001,United States,0,2020-12,1693.0
8904,102001,United States,0,2021-01,1701.0


In [25]:
def run_prophet(series):
    model = Prophet()
    model.fit(series)
    forecast = model.make_future_dataframe(periods=12, freq='MS')
    forecast = model.predict(forecast)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast['RegionName'] = series['RegionName'].iloc[0]
    forecast = forecast[['RegionName','ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].astype(float)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
    return forecast

In [26]:
f = run_prophet(series[0])
f.tail(12)

,RegionName,ds,yhat,yhat_lower,yhat_upper
86,United States,2021-03-01,1705.0,1695.0,1715.0
87,United States,2021-04-01,1706.0,1697.0,1716.0
88,United States,2021-05-01,1707.0,1698.0,1717.0
89,United States,2021-06-01,1708.0,1698.0,1719.0
90,United States,2021-07-01,1709.0,1699.0,1720.0
...,...,...,...,...,...
93,United States,2021-10-01,1713.0,1701.0,1725.0
94,United States,2021-11-01,1714.0,1701.0,1726.0
95,United States,2021-12-01,1715.0,1702.0,1729.0
96,United States,2022-01-01,1716.0,1701.0,1730.0


In [ ]:
for i in range(len(series)):
    f = run_prophet(series[i])
    f.to_csv('csv/rental_prediction.csv', mode='a', index='False')

In [ ]:
rental_predictions = pd.read_csv('csv/rental_prediction.csv')

In [ ]:
rent_long = rental_prediction['RegionName', 'ds', 'yhat']

In [ ]:
rental_long = rental_long.pivot_table(index = 'RegionName',
                                                          columns = 'monthly',
                                                          values = 'yhat')

In [ ]:
def rental_forecast_graph(city):

    city = [city]

    # Historical population data
    rental = pd.read_csv('csv/Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv')
    rental = rental[rental['RegionName'].isin(city)]
    rent_melt = rental.melt(id_vars= ['RegionID', 'RegionName', 'SizeRank'],var_name = 'ds', value_name = 'y')).reset_index(drop=True)

    # Predictions
    forecast = pd.read_csv('csv/rental_prediction.csv')
    predictions = forecast[forecast['RegionName'].isin(city)][9:]
    predictions['ds'] = predictions['ds']

    # Graph Data
    ax = population_melt.plot(x = 'ds', y = 'y', label='Observed', figsize= (10, 8))
    predictions[['ds', 'yhat']].plot(ax = ax, x = 'ds', y = 'yhat', label = "Forecast")

    # Fill to show upper and lower bounds
    # Graph predictions including the upper and lower bounds
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        name = 'Original',
        x = rent_melt['ds'],
        y = rent_melt['y'],
        fill = None,
        mode = 'lines',
        line_color = 'black',
        showlegend = True
    ))

    fig.add_trace(go.Scatter(
        name = 'Forecast',
        x = predictions['ds'],
        y = predictions['yhat'],
        fill = None,
        mode = 'lines',
        line_color = 'red',
        showlegend = True
    ))

    fig.add_trace(go.Scatter(
        name = 'Lower Bound',
        x = predictions['ds'],
        y = predictions['yhat_lower'],
        fill = None,
        mode = 'lines',
        line_color = 'gray'
    ))

    fig.add_trace(go.Scatter(
        name = 'Upper Bound',
        x = predictions['ds'],
        y = predictions['yhat_upper'],
        fill='tonexty',
        mode='lines',
        line_color = 'gray'
    ))

    # Edit the layout
    fig.update_layout({
        'autosize':True,
        'title': f'{location[0]} Population Forecast',
        'title_x': 0.5,
        'xaxis_title': 'Monthly',
        'yaxis_title': 'Population'
        })

    fig.update_yaxes(automargin = True)
    fig.update_xaxes(automargin = True, nticks=20)

    fig.show()

    return fig.to_json()
